In [2]:
import networkx as nx
import os
import random

# Asegura que las carpetas necesarias existen
def crear_estructura_directorios():
    base = "ARCHIVOS_REDES"
    subdirs = ["ER", "WS", "ER_parametros", "WS_parametros"]
    for sub in subdirs:
        os.makedirs(os.path.join(base, sub), exist_ok=True)

crear_estructura_directorios()

def contar_archivos_en_directorio(directorio):
    return len([f for f in os.listdir(directorio) if f.endswith(".txt")])


def guardar_lista_adyacencia(grafo, nombre_base, modelo, parametros):
    """
    Guarda el grafo en formato lista de adyacencia y los parámetros usados.

    Args:
        grafo (nx.Graph): Grafo a guardar.
        nombre_base (str): Nombre base para el archivo.
        modelo (str): 'ER' o 'WS'.
        parametros (dict): Parámetros a guardar en el archivo de parámetros.
    """
    carpeta_grafos = os.path.join("ARCHIVOS_REDES", modelo)
    carpeta_params = os.path.join("ARCHIVOS_REDES", f"{modelo}_parametros")

    i = contar_archivos_en_directorio(carpeta_grafos)
    nombre_grafo = f"{nombre_base}_{i}.txt"
    nombre_param = f"{nombre_base}_{i}.txt"

    # Guardar lista de adyacencia
    with open(os.path.join(carpeta_grafos, nombre_grafo), 'w') as f:
        for nodo in sorted(grafo.nodes()):
            vecinos = sorted(grafo.neighbors(nodo))
            for vecino in vecinos:
                f.write(f"{nodo} {vecino}\n")

    # Guardar parámetros
    with open(os.path.join(carpeta_params, nombre_param), 'w') as f:
        for clave, valor in parametros.items():
            f.write(f"{clave}: {valor}\n")


def generar_red_ER_conexa(N, k_prom, aceptancia_N, aceptancia_k, nombre_base="ER"):
    p = k_prom / (N - 1)

    while True:
        G = nx.erdos_renyi_graph(N, p)
        if not nx.is_connected(G):
            giant = max(nx.connected_components(G), key=len)
            G = G.subgraph(giant).copy()

        N_real = G.number_of_nodes()
        k_real = sum(dict(G.degree()).values()) / N_real

        if (abs(N_real - N) / N * 100 <= aceptancia_N and
                abs(k_real - k_prom) / k_prom * 100 <= aceptancia_k):
            parametros = {
                "N_ideal": N,
                "N_real": N_real,
                "k_prom_ideal": k_prom,
                "k_prom_real": round(k_real, 3)
            }
            guardar_lista_adyacencia(G, nombre_base, "ER", parametros)
            return G, N_real, k_real


def generar_red_WS(N, k, p, nombre_base="WS"):
    G = nx.watts_strogatz_graph(N, k, 0)
    feos = []
    guapos = []

    for edge in G.edges():
        if random.random() < p:
            feos.append(edge)
        else:
            guapos.append(edge)

    while feos:
        random.shuffle(feos)
        enlace0 = feos[0]
        nodo0_a, nodo0_b = enlace0
        encontrado = False

        for i in range(1, len(feos)):
            enlace1 = feos[i]
            nodo1_a, nodo1_b = enlace1
            if nodo0_a not in enlace1 and nodo0_b not in enlace1:
                nodos1 = [nodo1_a, nodo1_b]
                random.shuffle(nodos1)
                A = (nodo0_a, nodos1[0])
                B = (nodo0_b, nodos1[1])
                if A not in guapos and (A[1], A[0]) not in guapos and B not in guapos and (B[1], B[0]) not in guapos:
                    guapos.append(A)
                    guapos.append(B)
                    del feos[i]
                    del feos[0]
                    encontrado = True
                    break

        if not encontrado:
            guapos.append(feos[0])
            del feos[0]

    G_guapo = nx.Graph()
    G_guapo.add_nodes_from(range(N))
    G_guapo.add_edges_from(guapos)

    parametros = {
        "N": N,
        "k": k,
        "p": p
    }

    guardar_lista_adyacencia(G_guapo, nombre_base, "WS", parametros)
    return G_guapo


In [4]:
## ERDOS REYNI
## LO UNICO QUE HAY QUE TOCAR!!!!!!!!!!!!!!!
Numero_redes_creadas=20
N=1000
k=6
aceptancia_k=1 #aceptancia del 1% para el valor de K promedio
aceptancia_N=1 #aceptancia del 1% para el valor de N

for i in range(Numero_redes_creadas):
    generar_red_ER_conexa(N, k, aceptancia_N, aceptancia_k)



In [5]:
## WATS-STROTGTARZ
## LO UNICO QUE HAY QUE TOCAR!!!!!!!!!!!!!!!
Numero_redes_creadas=20
N=1000
k=4 
p=0.2

for i in range(Numero_redes_creadas):
    generar_red_WS(N, k, p)